In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls /content/drive/MyDrive/dataset/


test  train  validation


In [ ]:
import os

train_dir = '/content/drive/MyDrive/dataset/train'
val_dir = '/content/drive/MyDrive/dataset/validation'
test_dir = '/content/drive/MyDrive/dataset/test'

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)


In [ ]:
import os
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Path to image directories
train_dir = '/content/drive/MyDrive/dataset/train'  # Adjust the path
img_size = (150, 150)  # Resize images to 150x150

# Function to load and preprocess images
def load_and_preprocess_images(directory, img_size):
    images = []
    labels = []
    for label in ['healthy', 'infected']:  # Ensure 'healthy' and 'infected' are folder names
        label_dir = os.path.join(directory, label)
        for img_filename in os.listdir(label_dir):
            img_path = os.path.join(label_dir, img_filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load image in grayscale
            img_resized = cv2.resize(img, img_size)  # Resize image
            img_normalized = img_resized / 255.0  # Normalize pixel values
            images.append(img_normalized)
            labels.append(0 if label == 'healthy' else 1)  # 0 for healthy, 1 for infected
    return np.array(images), np.array(labels)

# Load and preprocess the images
images, labels = load_and_preprocess_images(train_dir, img_size)

# Reshape images to add a channel dimension (for grayscale)
images = np.expand_dims(images, axis=-1)

# Split data into training and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Check the shape of the data
print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")


Training data shape: (80, 150, 150, 1)
Validation data shape: (21, 150, 150, 1)


In [ ]:
# Example: Convert pixel values to temperatures
min_temp = 20  # Minimum temperature (assumption)
max_temp = 80  # Maximum temperature (assumption)

def pixel_to_temperature(img):
    # Normalize pixel values to temperature range
    temp_img = img * (max_temp - min_temp) / 255.0 + min_temp
    return temp_img


In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

# Path to the image directory
train_dir = '/content/drive/MyDrive/dataset/train'  # Update path as necessary
img_size = (150, 150)  # Resize images to 150x150 pixels

# Temperature normalization function
def pixel_to_temperature(img, min_temp, max_temp):
    return img * (max_temp - min_temp) / 255.0 + min_temp

# Function to load and preprocess thermal images with temperature normalization
def load_and_preprocess_images(directory, img_size, min_temp=20, max_temp=80):
    images = []
    labels = []
    for label in ['healthy', 'infected']:
        label_dir = os.path.join(directory, label)
        for img_filename in os.listdir(label_dir):
            img_path = os.path.join(label_dir, img_filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load image in grayscale
            img_resized = cv2.resize(img, img_size)  # Resize image to uniform size
            img_temp_normalized = pixel_to_temperature(img_resized, min_temp, max_temp)  # Convert to temperature values
            img_normalized = img_temp_normalized / max_temp  # Normalize temperature values to [0, 1]
            images.append(img_normalized)
            labels.append(0 if label == 'healthy' else 1)  # Assign labels (0 = healthy, 1 = infected)
    return np.array(images), np.array(labels)

# Load and preprocess images
images, labels = load_and_preprocess_images(train_dir, img_size)

# Reshape images to add channel dimension for CNN input
images = np.expand_dims(images, axis=-1)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Check the shape of the data
print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")


Training data shape: (80, 150, 150, 1)
Validation data shape: (21, 150, 150, 1)


In [ ]:
# Function to apply edge detection
def apply_edge_detection(img):
    edges = cv2.Canny(img.astype(np.uint8), 100, 200)  # Apply Canny edge detection
    return edges

# Applying edge detection to preprocessed images
def load_and_preprocess_images_with_edges(directory, img_size, min_temp=20, max_temp=80):
    images = []
    labels = []
    for label in ['healthy', 'infected']:
        label_dir = os.path.join(directory, label)
        for img_filename in os.listdir(label_dir):
            img_path = os.path.join(label_dir, img_filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img_resized = cv2.resize(img, img_size)
            img_temp_normalized = pixel_to_temperature(img_resized, min_temp, max_temp)
            img_normalized = img_temp_normalized / max_temp
            img_edges = apply_edge_detection(img_normalized)  # Apply edge detection
            images.append(img_edges)
            labels.append(0 if label == 'healthy' else 1)
    return np.array(images), np.array(labels)

# Use this updated function for loading data if you want edge detection


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Create a CNN model
def create_cnn_model(input_shape):
    model = models.Sequential()

    # Convolutional layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Flatten the output and feed into dense layers
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))

    # Output layer for binary classification
    model.add(layers.Dense(1, activation='sigmoid'))  # Sigmoid for binary classification

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Get input shape from training data
input_shape = X_train.shape[1:]

# Create the model
model = create_cnn_model(input_shape)

# Model summary
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 148, 148, 32)        │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 36992)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       4,735,104 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,827,905 (18.42 MB)

 Trainable params: 4,827,905 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Define image augmentation (optional)
train_datagen = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
                                   width_shift_range=0.2, height_shift_range=0.2,
                                   shear_range=0.15, horizontal_flip=True)

val_datagen = ImageDataGenerator()  # No augmentation for validation

# Train the model
history = model.fit(
    train_datagen.flow(X_train, y_train, batch_size=32),
    validation_data=val_datagen.flow(X_val, y_val),
    epochs=50
)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.5023 - loss: 0.7095 - val_accuracy: 0.5714 - val_loss: 0.6920
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.4677 - loss: 0.6937 - val_accuracy: 0.4286 - val_loss: 0.6941
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.5227 - loss: 0.6927 - val_accuracy: 0.4286 - val_loss: 0.6970
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.5344 - loss: 0.6913 - val_accuracy: 0.4286 - val_loss: 0.7114
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.5240 - loss: 0.6932 - val_accuracy: 0.4286 - val_loss: 0.7108
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.5031 - loss: 0.6981 - val_accuracy: 0.4286 - val_loss: 0.7013
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.5188 - loss: 0.6927 - val_accuracy: 0.4286 - val_loss: 0.7012
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.5292 - loss: 0.6920 - val_accuracy: 0.4286 - val_loss: 0.7016
Epoch 9/50
3/3 ━━━━━━━━━━━━

In [ ]:
# Evaluate the model
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

# Save the model
model.save('thermal_image_classifier.h5')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step - accuracy: 0.4286 - loss: 0.7029


Validation Loss: 0.7029162645339966
Validation Accuracy: 0.4285714328289032


In [ ]:
import os

test_img_path = '/path/to/test/image.jpg'  # Replace with your actual test image path

# Check if the file exists
if os.path.exists(test_img_path):
    print(f"Image found: {test_img_path}")
else:
    print(f"Image not found: {test_img_path}")


Image not found: /path/to/test/image.jpg


In [ ]:
test_img_path = '/content/drive/MyDrive/dataset/test/your_test_image.jpg'


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Directory where your images are stored
image_dir = '/content/drive/MyDrive/dataset/'

# Initialize the data generator (no separate classes, just loading)
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # Use 20% of data for validation

# Load training and validation data
train_generator = datagen.flow_from_directory(
    image_dir,
    target_size=(150, 150),  # You can adjust the size based on your image resolution
    batch_size=32,
    class_mode=None,  # No classes yet, since we don't have labels
    shuffle=True,
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    image_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode=None,  # No classes yet
    shuffle=True,
    subset='validation'
)


Found 202 images belonging to 3 classes.
Found 49 images belonging to 3 classes.


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def load_images_from_directory(directory, target_size=(150, 150)):
    images = []
    image_filenames = [f for f in os.listdir(directory) if f.endswith(('jpg', 'jpeg', 'png'))]
    for filename in image_filenames:
        img_path = os.path.join(directory, filename)
        img = load_img(img_path, target_size=target_size)
        img_array = img_to_array(img)
        images.append(img_array)
    return np.array(images)

# Load images from the directory
image_dir = '/content/drive/MyDrive/dataset/'  # Adjust to your path
train_images = load_images_from_directory(image_dir)

# Normalize the images (as we were doing with rescale=1./255)
train_images = train_images / 255.0

# Split into training and validation sets (80% train, 20% validation)
split_idx = int(0.8 * len(train_images))
train_data = train_images[:split_idx]
val_data = train_images[split_idx:]


In [ ]:
# Train the autoencoder with the loaded images
autoencoder.fit(train_data, train_data,  # Input and output are the same
                epochs=50,
                batch_size=32,
                validation_data=(val_data, val_data))


In [ ]:
# Check the shape of the train and validation data
print(f"Train data shape: {train_data.shape}")
print(f"Validation data shape: {val_data.shape}")


In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

def load_images_from_directory(directory, target_size=(150, 150)):
    images = []
    image_filenames = [f for f in os.listdir(directory) if f.endswith(('jpg', 'jpeg', 'png'))]
    print(f"Found {len(image_filenames)} images in {directory}")

    for filename in image_filenames:
        img_path = os.path.join(directory, filename)
        print(f"Loading image: {img_path}")

        try:
            img = load_img(img_path, target_size=target_size)  # Resize images
            img_array = img_to_array(img)  # Convert to array (150, 150, 3)
            images.append(img_array)
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")

    return np.array(images)

# Load images from the directory
train_images = load_images_from_directory(image_dir)

# Check if any images were loaded
print(f"Loaded {len(train_images)} images.")

# Normalize the images (rescale pixel values)
if len(train_images) > 0:
    train_images = train_images / 255.0

    # Split into training and validation sets (80% train, 20% validation)
    split_idx = int(0.8 * len(train_images))
    train_data = train_images[:split_idx]
    val_data = train_images[split_idx:]

    print(f"Train data shape: {train_data.shape}")
    print(f"Validation data shape: {val_data.shape}")
else:
    print("No images were loaded.")
